In [17]:
import pandas as pd
import numpy as np
import math
from math import sqrt
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.metrics import accuracy_score, log_loss
import torch
import torch.nn as nn
from datetime import timedelta

'''
Note: 2 way to index an element in pandas df
By Index:
- df.Time[row]
- df.loc[row, 'Time']

By location:
- df.iloc[1, 1]: iloc only take interger

'''
def calculateTimeInterval(missing_data):
    df = missing_data.copy()
    df['TimeInterval'] = (df['CompleteTimestamp'] - df['CompleteTimestamp'].shift(1))
    df.loc[0, 'TimeInterval'] = 0
    return df

def calculateDuration(missing_data):
    df = missing_data.copy()
    df['Duration'] = (df['CompleteTimestamp'] - df['CompleteTimestamp'].iloc[0])
    df['Duration'].iloc[0] = 0
    return df

def calculateCumTimeInterval(missing_data):
    df = missing_data.copy()
    df['CumTimeInterval'] = (df['CompleteTimestamp'] - df['CompleteTimestamp'].iloc[0])
    return df

def calculateCaseTimeInterval(missing_data):
    df = missing_data.copy()
    df['CaseTimeInterval'] = np.nan

    current_point = {}
    current_point[df.loc[0, 'CaseID']] = df.loc[0, 'CompleteTimestamp']

    for i in range(1, df.shape[0]):
        if df.loc[i, 'CaseID'] != df.loc[i-1, 'CaseID']:
            df.loc[i, 'CaseTimeInterval'] = (df.loc[i, 'CompleteTimestamp'] - current_point[df.loc[i, 'CaseID']-1]).total_seconds()
            current_point[df.loc[i, 'CaseID']] = df.loc[i, 'CompleteTimestamp']
            
    return df

def convert2seconds(x):
    x = x.total_seconds()
    return x


def minmaxScaler(caseid, df_case, missing_df_case):
    epsilon = 0.1
    missing_case_storage = {}
    missing_case_storage[caseid] = {}
    
    temp = df_case.copy()
    missing_temp = missing_df_case.copy()
    
    temp['NormalizedTime'] = temp['CumTimeInterval'].copy()
    missing_temp['NormalizedTime'] = missing_temp['CumTimeInterval'].copy()
    
    min_val = temp['CumTimeInterval'].min()
    max_val = temp['CumTimeInterval'].max()
    
    missing_min_val = missing_temp['CumTimeInterval'].min()
    missing_max_val = missing_temp['CumTimeInterval'].max()
    missing_case_storage[caseid]['missing_min_val'] = missing_min_val
    missing_case_storage[caseid]['missing_max_val'] = missing_max_val
    
    for row in range(temp.shape[0]):
        #scale complete df
        temp.iloc[row, temp.columns.get_loc('NormalizedTime')] = (temp.iloc[row, temp.columns.get_loc('CumTimeInterval')] - min_val)/(max_val-min_val+epsilon)
        
        #scale missing df
        missing_temp.iloc[row, missing_temp.columns.get_loc('NormalizedTime')] = (missing_temp.iloc[row, missing_temp.columns.get_loc('CumTimeInterval')] - missing_min_val)/(missing_max_val-missing_min_val+epsilon)  
    return temp, missing_temp, missing_case_storage


def OHE(df, categorical_variables):
    for i in categorical_variables:
        enc_df = pd.get_dummies(df, columns=categorical_variables, drop_first=False)
    return enc_df

def findLongestLength(groupByCase):
    '''This function returns the length of longest case'''
    #groupByCase = data.groupby(['CaseID'])
    maxlen = 1
    for case, group in groupByCase:
        temp_len = group.shape[0]
        if temp_len > maxlen:
            maxlen = temp_len
    return maxlen

def padwithzeros(vector, maxlen):
    '''This function returns the (maxlen, num_features) vector padded with zeros'''
    npad = ((maxlen-vector.shape[0], 0), (0, 0))
    padded_vector = np.pad(vector, pad_width=npad, mode='constant', constant_values=0)
    return padded_vector

def getInput(groupByCase, cols, maxlen):
    full_list = []
    for case, data in groupByCase:
        temp = data.to_numpy()#data.as_matrix(columns=cols)
        temp = padwithzeros(temp, maxlen)
        full_list.append(temp)
    inp = np.array(full_list)
    return inp

def getMeanVar(array, idx=0):
    temp_array = [a[idx] for a in array if not np.isnan(a[idx])]
    mean_val = np.mean(temp_array)
    var_val = np.var(temp_array)
    return mean_val, var_val

def getProbability(recon_test):
    '''This function takes 3d tensor as input and return a 3d tensor which has 
    probabilities for classes of categorical variable'''
    softmax = nn.Softmax()
    #recon_test = recon_test.cpu() #moving data from gpu to cpu for full evaluation
    
    for i in range(recon_test.size(0)):
        cont_values = recon_test[i, :, 0].contiguous().view(recon_test.size(1),1) #(35,1)
        #softmax_values = softmax(recon_test[i, :, 1:])
        softmax_v = softmax()
        softmax_values = softmax_v.fit(recon_test[i, :, 1:])
        if i == 0:
            recon = torch.cat([cont_values, softmax_values], 1)
            recon = recon.contiguous().view(1,recon_test.size(1), recon_test.size(2)) #(1, 35, 8)
        else:
            current_recon = torch.cat([cont_values, softmax_values], 1)
            current_recon = current_recon.contiguous().view(1,recon_test.size(1), recon_test.size(2)) #(1, 35, 8)
            recon = torch.cat([recon, current_recon], 0)
    return recon

def convert2df(predicted_tensor, pad_matrix, cols, test_row_num):
    '''
    This function converts a tensor to a pandas dataframe
    Return: Dataframe with columns (NormalizedTime, PredictedActivity)

    - predicted_tensor: recon
    - df: recon_df_w_normalized_time
    '''
    print("WORKS")
    #predicted_tensor = getProbability(predicted_tensor) #get probability for categorical variables
    predicted_array = predicted_tensor.data.cpu().numpy() #convert to numpy array
    
    #Remove 0-padding
    temp_array = np.array(predicted_array) * np.array(pad_matrix)
    temp_array = temp_array.reshape(predicted_array.shape[0]*predicted_array.shape[1], predicted_array.shape[2])
    temp_array = temp_array[np.any(temp_array != 0, axis=1)]
    
    #check number of row of df
    if temp_array.shape[0] == test_row_num:
        #print('Converting tensor to dataframe...')
        df = pd.DataFrame(temp_array, columns=cols)
        activity_list = [i for i in cols if i!='NormalizedTime']
        df['PredictedActivity'] = df[activity_list].idxmax(axis=1) #get label
        #df['PredictedActivity'] = df['PredictedActivity'].apply(lambda x: x.split('_')[1]) #remove prefix
        df['PredictedActivity'] = df['PredictedActivity'].apply(lambda x: x[9:]) #remove prefix Activity_
        df = df.drop(activity_list, axis=1)
        #print('Done!!!')
    return df

def inversedMinMaxScaler(caseid, min_max_storage, recon_df_w_normalized_time_case):
    epsilon = 0.1
    
    temp = recon_df_w_normalized_time_case.copy()
    temp['PredictedCumTimeInterval'] = recon_df_w_normalized_time_case['NormalizedTime'].copy()
    
    #should check for nan values here
    #min_val = min_max_storage[caseid]['missing_min_val']
    #max_val = min_max_storage[caseid]['missing_max_val']
    min_val, max_val = findValidMinMax(caseid, min_max_storage)

    for row in range(temp.shape[0]):
        temp.iloc[row, temp.columns.get_loc('PredictedCumTimeInterval')] = min_val + temp.iloc[row, temp.columns.get_loc('NormalizedTime')]*(max_val-min_val+epsilon)
        
    return temp

def findValidMinMax(caseid, min_max_storage):
    min_val_before = 0
    max_val_before= 0
    min_val_after = 0
    max_val_after = 0
    min_val = 0
    max_val = 0
    
    if caseid == len(min_max_storage):
        for i in range(caseid):
            min_val = min_max_storage[caseid-i]['missing_min_val']
            max_val = min_max_storage[caseid-i]['missing_max_val']
            if not np.isnan(min_val) and not np.isnan(max_val):
                break
    else:
        for i in range(caseid):
            min_val_before = min_max_storage[caseid-i]['missing_min_val']
            max_val_before = min_max_storage[caseid-i]['missing_max_val']
            if not np.isnan(min_val_before) and not np.isnan(max_val_before):
                break
    
        for j in range(len(min_max_storage) - caseid+1):
            min_val_after = min_max_storage[caseid+j]['missing_min_val']
            max_val_after = min_max_storage[caseid+j]['missing_max_val']
            if not np.isnan(min_val_after) and not np.isnan(max_val_after):
                break
        min_val = (min_val_before+min_val_after)/2
        max_val = (max_val_before+max_val_after)/2
    return min_val, max_val


def getDfWithTime(recon_df_w_normalized_time, missing_true_test, min_max_storage):
    temp = recon_df_w_normalized_time.copy()
    temp['CaseID'] = missing_true_test['CaseID'].copy()
    recon_groupByCase = temp.groupby(['CaseID'])
    recon_df_w_time = pd.DataFrame(columns=list(temp)+['PredictedCumTimeInterval'])
    
    for caseid, data_case in recon_groupByCase:
        temp_case = inversedMinMaxScaler(caseid, min_max_storage, data_case)
        recon_df_w_time = recon_df_w_time.append(temp_case)
    return recon_df_w_time



def getnanindex(missing_true_df):
    nan_time_index = []
    nan_activity_index = []
    for row in range(missing_true_df.shape[0]):
        if np.isnan(missing_true_df.CumTimeInterval[row]):
            nan_time_index.append(row)

        if not type(missing_true_df.Activity[row]) == str:
            nan_activity_index.append(row)
    return nan_time_index, nan_activity_index

def getSubmission(recon_df_w_time, missing_true_test, complete_true_test, first_timestamp):
    temp = pd.DataFrame(columns=['CaseID', 'TrueActivity', 'PredictedActivity', 'TrueTime', 'PredictedTime'])
    temp['CaseID'] = missing_true_test['CaseID'].copy()
    
    #ground truth
    temp['TrueActivity'] = complete_true_test['Activity'].copy()
    temp['TrueTime'] = complete_true_test['CumTimeInterval'].copy()
    temp['TrueCompleteTimestamp'] = complete_true_test['CompleteTimestamp'].copy()

    #predicted activity
    temp['PredictedActivity'] = missing_true_test['Activity'].copy()
    temp['PredictedTime'] = missing_true_test['CumTimeInterval'].copy()
    temp['PredictedCompleteTimestamp'] = missing_true_test['CompleteTimestamp'].copy()

    for row in range(temp.shape[0]):
        if pd.isnull(temp.loc[row, 'PredictedActivity']):
            temp.loc[row, 'PredictedActivity'] = recon_df_w_time.loc[row, 'PredictedActivity']
        if pd.isnull(temp.loc[row, 'PredictedTime']):
            temp.loc[row, 'PredictedTime'] = recon_df_w_time.loc[row, 'PredictedCumTimeInterval']
            temp.loc[row, 'PredictedCompleteTimestamp'] = first_timestamp+timedelta(seconds=recon_df_w_time.loc[row, 'PredictedCumTimeInterval'])
    return temp

def fixTime(recon_df_w_time):
    groupByCase = recon_df_w_time.groupby(['CaseID'])
    temp = pd.DataFrame(columns=list(recon_df_w_time))

    for caseid, data_case in groupByCase:
        for row in range(1, len(data_case)):
            current = data_case.iloc[row, data_case.columns.get_loc('PredictedTime')]
            previous = data_case.iloc[row-1, data_case.columns.get_loc('PredictedTime')]
            if current < previous:
                data_case.iloc[row, data_case.columns.get_loc('PredictedTime')] = previous
                data_case.iloc[row, data_case.columns.get_loc('PredictedCompleteTimestamp')] = data_case.iloc[row-1, data_case.columns.get_loc('PredictedCompleteTimestamp')]
        temp = temp.append(data_case)
    return temp


def evaluation(submission_df, nan_time_index, nan_activity_index, show=False):
    #eval Time
    true_time = submission_df.loc[nan_time_index, 'TrueTime']
    predicted_time = submission_df.loc[nan_time_index, 'PredictedTime']
    mae_time = mean_absolute_error(true_time, predicted_time)
    rmse_time = sqrt(mean_squared_error(true_time, predicted_time))
    
    #eval Activity
    true_activity = submission_df.loc[nan_activity_index, 'TrueActivity']
    predicted_activity = submission_df.loc[nan_activity_index, 'PredictedActivity']
    acc = accuracy_score(true_activity, predicted_activity)
    
    if show==True: 
        print('Number of missing Time: {}'.format(len(nan_time_index)))
        print('Mean Absolute Error: {:.4f} day(s)'.format(mae_time/86400))
        print('Root Mean Squared Error: {:.4f} day(s)'.format(rmse_time/86400))
        
        print('Number of missing Activity: {}'.format(len(nan_activity_index)))
        print('Accuracy: {:.2f}%'.format(acc*100))
    return mae_time, rmse_time, acc


In [22]:
import os, sys
import argparse
import pandas as pd
import numpy as np
import pickle

from dateutil.parser import parse
from datetime import datetime
import time


pd.options.mode.chained_assignment = None #to run loop quicker without warnings

args_parser = argparse.ArgumentParser(description='induce missing data')
args_parser.add_argument('-n', '--name', default="small_log", metavar='<data_name>', help='Name of input file')
args_parser.add_argument('-d', '--data_dir', default='../data/', help='data dir')
args_parser.add_argument('--nan_pct', default=0.3, type=float, help='Nan percentage')
args_parser.add_argument('--train_pct', default=0.6, type=float, help='Train percentage')
args_parser.add_argument('--val_pct', default=0.2, type=float, help='Validate percentage')
args = args_parser.parse_args()
args.data_file = args.name + '.csv'
args.input_dir = '../input/small_log/'


#name = 'bpi_2012'
#name = 'bpi_2013'
#name = 'Road_Traffic_Fine_Management_Process'
'''
args = {
    'data_dir': '../data/',
    'data_file': name + '.csv',
    'input_dir': '../input/{}/'.format(name),  
    'nan_pct': 0.3,
    'train_pct': 0.6,
    'val_pct': 0.2,
}

args = argparse.Namespace(**args)
'''

file_name = os.path.join(args.input_dir, 'parameters_{}.pkl'.format(args.nan_pct))
with open(file_name, 'rb') as f:
    most_frequent_activity = pickle.load(f)
    first_timestamp = pickle.load(f)
    avai_instance = pickle.load(f)
    nan_instance = pickle.load(f)
    train_size = pickle.load(f)
    val_size = pickle.load(f)
    test_size = pickle.load(f)
    train_row_num = pickle.load(f)
    val_row_num = pickle.load(f)
    test_row_num = pickle.load(f)
    


usage: ipykernel_launcher [-h] [-n <data_name>] [-d DATA_DIR]
                          [--nan_pct NAN_PCT] [--train_pct TRAIN_PCT]
                          [--val_pct VAL_PCT]
ipykernel_launcher: error: unrecognized arguments: --ip=127.0.0.1 --stdin=9023 --control=9021 --hb=9020 --Session.signature_scheme="hmac-sha256" --Session.key=b"30e3a4c5-928f-42a9-902d-e2385daff0ed" --shell=9022 --transport="tcp" --iopub=9024 --f=/tmp/tmp-15580dhmTEGCjUQsl.json


SystemExit: 2

In [21]:

file_name = os.path.join(args.input_dir, 'parameters_{}.pkl'.format(args.nan_pct))
with open(file_name, 'rb') as f:
    most_frequent_activity = pickle.load(f)
    first_timestamp = pickle.load(f)
    avai_instance = pickle.load(f)
    nan_instance = pickle.load(f)
    train_size = pickle.load(f)
    val_size = pickle.load(f)
    test_size = pickle.load(f)
    train_row_num = pickle.load(f)
    val_row_num = pickle.load(f)
    test_row_num = pickle.load(f)
    

NameError: name 'args' is not defined

In [2]:

#Load data
complete_df_full_name = 'complete_df_full_{}.csv'.format(args.nan_pct)
missing_df_full_name = 'missing_df_full_{}.csv'.format(args.nan_pct)
print('Loading data:')
print(args.name)
print(complete_df_full_name)
print(missing_df_full_name)

df_name = os.path.join(args.input_dir, complete_df_full_name)
df = pd.read_csv(df_name)

missing_df_name = os.path.join(args.input_dir, missing_df_full_name)
missing_df = pd.read_csv(missing_df_name)



NameError: name 'args' is not defined

In [3]:
#Preprocess data
print('Processing data...')
groupByCase = df.groupby(['CaseID'])

groupByCase = df.groupby(['CaseID'])
missing_groupByCase = missing_df.groupby(['CaseID'])

normalized_complete_df = pd.DataFrame(columns=list(df)+['NormalizedTime'])
normalized_missing_df = pd.DataFrame(columns=list(df)+['NormalizedTime'])
min_max_storage = {}

for i, j in zip(groupByCase, missing_groupByCase):
    temp, missing_temp, missing_case_storage = minmaxScaler(i[0], i[1], j[1])
    normalized_complete_df = normalized_complete_df.append(temp)
    normalized_missing_df = normalized_missing_df.append(missing_temp)
    min_max_storage.update(missing_case_storage)


cat_var = ['Activity']



Processing data...


NameError: name 'df' is not defined

In [4]:

# OHE: get k dummies out of k categorical levels (drop_first=False)
enc_complete_df = OHE(normalized_complete_df, cat_var)
enc_missing_df = OHE(normalized_missing_df, cat_var)

print('Getting masks...')

c_df = enc_complete_df.copy()
m_df = enc_missing_df.copy()
enc_complete_df_w_normalized_time = c_df.drop(['CompleteTimestamp', 'CumTimeInterval'], axis=1)
enc_missing_df_w_normalized_time = m_df.drop(['CompleteTimestamp', 'CumTimeInterval'], axis=1)

c_df = enc_complete_df.copy()
m_df = enc_missing_df.copy()
enc_complete_df_w_time = c_df.drop(['CompleteTimestamp', 'NormalizedTime'], axis=1)
enc_missing_df_w_time = m_df.drop(['CompleteTimestamp', 'NormalizedTime'], axis=1)

avai_index_df = enc_missing_df_w_time.copy()
nan_index_df = enc_missing_df_w_time.copy()


NameError: name 'OHE' is not defined

In [5]:

#mask for Time
print('Mask for Time')
for row in range(enc_missing_df_w_time.shape[0]):
    if np.isnan(enc_missing_df_w_time.loc[row, 'CumTimeInterval']): # if nan Time
        avai_index_df.loc[row, 'CumTimeInterval'] = 0
        nan_index_df.loc[row, 'CumTimeInterval'] = 1
    else:
        avai_index_df.loc[row, 'CumTimeInterval'] = 1
        nan_index_df.loc[row, 'CumTimeInterval'] = 0
 

Mask for Time


NameError: name 'enc_missing_df_w_time' is not defined

In [ ]:
       
#mask for Activity
print('Mask for Activity')
for row in range(enc_missing_df_w_time.shape[0]):
    if np.any(enc_missing_df_w_time.iloc[row,2:]>0): #if avai Time
        avai_index_df.iloc[row, 2:] = 1
        nan_index_df.iloc[row, 2:] = 0
    else:
        avai_index_df.iloc[row, 2:] = 0
        nan_index_df.iloc[row, 2:] = 1
        
pad_index_df = enc_complete_df.copy()
cols = [x for x in list(pad_index_df) if x != 'CaseID']
pad_index_df.loc[:, cols] = 1

enc_missing_df_w_normalized_time.fillna(0, inplace=True)
enc_missing_df_w_time.fillna(0, inplace=True)

enc_complete_w_normalized_time_groupByCase = enc_complete_df_w_normalized_time.groupby(['CaseID'])
enc_missing_w_normalized_time_groupByCase = enc_missing_df_w_normalized_time.groupby(['CaseID'])

enc_complete_w_time_groupByCase = enc_complete_df_w_time.groupby(['CaseID'])
enc_missing_w_time_groupByCase = enc_missing_df_w_time.groupby(['CaseID'])

avai_index_df_groupByCase = avai_index_df.groupby(['CaseID'])
nan_index_df_groupByCase = nan_index_df.groupby(['CaseID'])
pad_index_df_groupByCase = pad_index_df.groupby(['CaseID'])

maxlen = findLongestLength(groupByCase)
print('Length of longest case: {}'.format(maxlen))

cols_w_time = [i for i in list(enc_complete_df_w_time) if i != 'CaseID']
cols_w_normalized_time = [i for i in list(enc_complete_df_w_normalized_time) if i != 'CaseID']

vectorized_complete_df_w_normalized_time = getInput(enc_complete_w_normalized_time_groupByCase, cols_w_normalized_time, maxlen)
vectorized_missing_df_w_normalized_time = getInput(enc_missing_w_normalized_time_groupByCase, cols_w_normalized_time, maxlen)

vectorized_complete_df_w_time = getInput(enc_complete_w_time_groupByCase, cols_w_time, maxlen)
vectorized_missing_df_w_time = getInput(enc_missing_w_time_groupByCase, cols_w_time, maxlen)

vectorized_avai_index_df = getInput(avai_index_df_groupByCase, cols_w_time, maxlen)
vectorized_nan_index_df = getInput(nan_index_df_groupByCase, cols_w_time, maxlen)
vectorized_pad_index_df = getInput(pad_index_df_groupByCase, cols_w_time, maxlen)


complete_matrix_w_normalized_time = vectorized_complete_df_w_normalized_time
missing_matrix_w_normalized_time = vectorized_missing_df_w_normalized_time

avai_matrix = vectorized_avai_index_df
nan_matrix = vectorized_nan_index_df
pad_matrix = vectorized_pad_index_df


print('Saving preprocessed data...')
preprocessed_data_name = os.path.join(args.input_dir, 'preprocessed_data_full_{}.pkl'.format(args.nan_pct))
with open(preprocessed_data_name, 'wb') as f:
    pickle.dump(min_max_storage, f, protocol=2)
    pickle.dump(complete_matrix_w_normalized_time, f, protocol=2)
    pickle.dump(missing_matrix_w_normalized_time, f, protocol=2)
    pickle.dump(avai_matrix, f, protocol=2)
    pickle.dump(nan_matrix, f, protocol=2)
    pickle.dump(pad_matrix, f, protocol=2)
    pickle.dump(cols_w_time, f, protocol=2)
    pickle.dump(cols_w_normalized_time, f, protocol=2)
    
print('Finish!!!')